# Planet : Amazon space

In [ ]:
import os
import torch
import pandas as pd
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms.functional
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torchvision.transforms.functional as F

from torchinfo import summary

from tqdm import tqdm

In [ ]:
data_folder = '../IPEO_Planet_project'
if not os.path.exists(data_folder):
    data_folder = input("Enter the data folder path: ")
    assert os.path.exists(data_folder), "I did not find the folder at, "+str(data_folder)

# Loading the dataset
### Understanding the labels

In [ ]:
labels_dt = pd.read_csv(f'{data_folder}/train_labels.csv',dtype=str)
tags = pd.Series(labels_dt['tags'].str.split(expand=True).stack().unique())
#print(F"{len(tags)} unique tags are: \n{tags.tolist()} ")
shape1 = labels_dt.shape
for tag in tags:
     a = [(tag in i.split()) for i in labels_dt['tags']]
     new_col = np.zeros(len(a),dtype=int)+a
     #print(new_col.shape)
     labels_dt[tag]=new_col
print(f" \n Shape of the initial dataset {shape1} and final {labels_dt.shape}")
labels_dt.head()

## Making sure the dataset is clean:

In [ ]:
def checking_folder(data_folder = data_folder):
    corrupted_files = []
    i = 0
    for img_id in tqdm(labels_dt['image_name']):
        #if i % 1000 == 0:
        #    print(f"image {i}")
        img_name = os.path.join(f'{data_folder}/train-jpg', f'{img_id}.jpg')
        img = imread(img_name)
        if img is None:
            print(f'{img_id} is corrupt...')
            corrupted_files.append(img_id)
        img = None
        i = i + 1
    return corrupted_files

#checking_folder()

## Custom made Class for multilabel classification dataset

See this tutorial for help:
<https://pytorch.org/tutorials/beginner/data_loading_tutorial.html>

In [ ]:
class AmazonSpaces(Dataset):
    """Amazon aerial image dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with labels.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = pd.read_csv(csv_file)

        self.tags = self.labels['tags'].str.split(expand=True).stack().unique()
        for tag in self.tags:
            a = [(tag in i.split()) for i in self.labels['tags']]
            self.labels[tag]=np.zeros(len(a),dtype=int)+a

        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.labels.iloc[idx, 0])
        img_name = f'{img_name}.jpg'
        image = imread(img_name)
        #image = torch.from_numpy(image)

        if self.transform:
            image = self.transform(image)

        labels = self.labels.loc[idx, self.tags].to_numpy(dtype=np.float64)
        #labels = torch.from_numpy(labels)
        #other output
        sample = {'image': image, 'labels': labels}

        #print(type(image), type(labels))
        return sample

In [ ]:
Amazon_dataset = AmazonSpaces(csv_file=f'{data_folder}/train_labels.csv',
                                    root_dir=f'{data_folder}/train-jpg')

## Having some fun with transforms
Just testing some transforms and how to implement into dataset
First creating and Adjust_Saturation class to be able to apply adjust_saturation the same way as the other transforms class

In [ ]:
class AdjustSaturation(object):
    """Adjust the saturation of a tensor image.
    Args:
        saturation factor (float): if 0 -> black and white, if 1 -> same as the input
    """

    def __init__(self, saturation_factor):
        assert isinstance(saturation_factor, (int,float))
        self.saturation_factor = saturation_factor

    def __call__(self, img):
        new_tensor = transforms.functional.adjust_saturation(img, self.saturation_factor)

        return new_tensor

Test_transforms = transforms.Compose([transforms.ToTensor(), transforms.CenterCrop(256),transforms.RandomAutocontrast(p=1), AdjustSaturation(1.2)])

transformed_dataset = AmazonSpaces(csv_file=f'{data_folder}/train_labels.csv',
                                    root_dir=f'{data_folder}/train-jpg', transform=Test_transforms)

## Visualizing the two datasets:

In [ ]:
plt.figure(figsize=(12,10))
N=3
for i in range(N+1):
    #Image from original direct dataset
    #image, labels = Amazon_dataset[i]
    sample = Amazon_dataset[i]
    labels = sample['labels']
    image = sample['image']


    ax = plt.subplot(2, N+1, i + 1)
    plt.tight_layout()
    ax.set_title(f' {tags[labels==1]}')
    ax.axis('off')
    img = F.to_pil_image(image)
    ax.imshow(img)

    #Images from transformed_dataset

    sample_tf = transformed_dataset[i]
    labels_tf = sample_tf['labels']
    image_tf = sample_tf['image']

    ax = plt.subplot(2, N+1, i + 5)
    plt.tight_layout()
    ax.set_title(f' {tags[labels_tf==1]}')
    ax.axis('off')
    img = F.to_pil_image(image_tf)
    ax.imshow(img)

plt.show()

# Into Dataloader

In [ ]:
train_loader = DataLoader(transformed_dataset, batch_size=64,
                        shuffle=True, num_workers=0)

## Testing with plotting

In [ ]:
# Helper function to show a batch
def show_4_image_in_batch(sample_batched, tags = tags):
    images_batch, labels = sample_batched['image'], sample_batched['labels'] #old class output
    #images_batch, labels = sample_batched

    fig, axs = plt.subplots(1, 4)
    for i in range(4):
        img = F.to_pil_image(images_batch[i])
        axs[i].imshow(img)
        ids = labels[i,:].numpy()
        axs[i].set_title(f'#{i}:\n {tags[ids == 1]}')
    fig.set_figheight(10)
    fig.set_figwidth(12)
    plt.tight_layout()
    plt.show()

In [ ]:
for i_batch, sample_batched in enumerate(train_loader):
    print(f"Batch # {i_batch} with sizes: {sample_batched['image'].size(), sample_batched['labels'].size()}")
    # observe 4th batch and stop.
    if i_batch == 3:
        show_4_image_in_batch(sample_batched)
        break

# Multi-Layer convulotional network
## Model
Function of interest:
* torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
* torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
* torch.nn.BatchNorm1d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)

In [ ]:
class MultiLayerCNN(nn.Module):
    def __init__(self):
        super(MultiLayerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size = 5) #Input is a 3 plane 256x256 tensor (RBB) -> output 10 planes of 254x254
        self.pool_max = nn.MaxPool2d(2, 2) #output of dim-2 x dim-2
        self.conv2 = nn.Conv2d(10, 20, 5) #input 10 planes 127x127, output 20 planes of 125x125
        self.pool_avg = nn.AvgPool2d(4,4)
        self.fc = nn.Linear(20*27*27,17) #single dense layer for the network
        self.batchNorm = nn.BatchNorm1d(256)
        self.loss = nn.BCELoss()
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.batchNorm(x)
        #print(f"step 1 : {x.shape}")
        x = self.pool_max(nn.functional.relu(self.conv1(x)))
        #print(f"step 2 : {x.shape}")
        x = self.pool_avg(x)
        #print(f"step 3 : {x.shape}")
        x = nn.functional.relu(self.conv2(x))
        #print(f"step 4 : {x.shape}")
        x = x.view(-1, 20*27*27)
        #print(f"step 5 : {x.shape}")
        x = self.fc(x)
        return x

In [ ]:
net = MultiLayerCNN()
summary(net,input_size=(3,256,256))

## Training, Validation and accuracy functions

In [ ]:
def validate(net, dataloader,loss_fn=nn.BCELoss()):
    net.eval()
    count,acc,loss = 0,0,0
    with torch.no_grad():
        for features,labels in dataloader:
            out = net(features)
            loss += loss_fn(out,labels)
            pred = torch.max(out,1)[1]
            acc += (pred==labels).sum()
            count += len(labels)
    return loss.item()/count, acc.item()/count

In [ ]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = nn.BCELoss()):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count = 0,0,0
    for features,labels in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return total_loss.item()/count, acc.item()/count

In [ ]:
def train(net,train_loader,test_loader,optimizer=None,lr=0.01,epochs=2,loss_fn=nn.BCELoss()):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    res = { 'train_loss' : [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    for ep in range(epochs):
        tl,ta = train_epoch(net,train_loader,optimizer=optimizer,lr=lr,loss_fn=loss_fn)
        vl,va = validate(net,test_loader,loss_fn=loss_fn)
        print(f"Epoch {ep:2}, Train acc={ta:.3f}, Val acc={va:.3f}, Train loss={tl:.3f}, Val loss={vl:.3f}")
        res['train_loss'].append(tl)
        res['train_acc'].append(ta)
        res['val_loss'].append(vl)
        res['val_acc'].append(va)
    return res

# Full model running
Ideas from :
<https://learnopencv.com/multi-label-image-classification-with-pytorch-image-tagging/>
## 1) Putting the model on the gpu

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

In [ ]:
model = MultiLayerCNN().to(device)

## 2) Getting the different datasets

In [ ]:
train_csv = data_folder+'/train.csv'
validation_csv = data_folder+'/validation.csv'
test_csv = data_folder+'/test.csv'

In [ ]:
Transform_choice = transforms.Compose([transforms.ToTensor(), transforms.CenterCrop(256),transforms.RandomAutocontrast(p=1), AdjustSaturation(1.2)])

In [ ]:
train_dataset = AmazonSpaces(csv_file=train_csv,
                                    root_dir=f'{data_folder}/train-jpg', transform=Transform_choice)
validation_dataset = AmazonSpaces(csv_file=validation_csv,
                                    root_dir=f'{data_folder}/train-jpg', transform=Transform_choice)
test_dataset = AmazonSpaces(csv_file=test_csv,
                                    root_dir=f'{data_folder}/train-jpg', transform=Transform_choice)

## 3) Wrapping into the different dataloaders

In [ ]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

## 4) Choice of Criterion and Opitimizer

In [ ]:
criterion = nn.BCEWithLogitsLoss

# Saving Models

In [ ]:
torch.save(model.state_dict(), "data/model_classification.pth")
print("Saved PyTorch Model State to model.pth")